HomeWork1

In [27]:
import cv2
import numpy as np

In [28]:
def nothing(x):
    pass

In [29]:
drawing = False  # true if mouse is pressed
mode = True  # if True, draw rectangle. Press 'm' to toggle to curve
ix, iy = -1, -1
font = cv2.FONT_HERSHEY_SIMPLEX
click_position = None  # 클릭한 좌표
release_position = None  # 마우스를 뗀 좌표
alpha = 1.0  # 초기 투명도 값 (완전 불투명)
rect_start = None  # 사각형 시작 좌표
rect_end = None  # 사각형 끝 좌표
img = cv2.imread('Everest_North_Face_toward_Base_Camp_Tibet_Luca_Galuzzi_2006.jpg')  # 이미지 파일 경로 확인
img_original = img.copy()  # 원본 이미지를 복사하여 저장

# mouse callback function
def draw_shape(event, x, y, flags, param):
    global ix, iy, drawing, mode, img, click_position, release_position, rect_start, rect_end
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y
        click_position = (ix, iy)  # 클릭한 좌표 저장
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            img = img_original.copy()  # 매번 원본 이미지를 복사
            if mode:
                cv2.rectangle(img, (ix, iy), (x, y), (0, 0, 255), -1)  # 사각형 그리기
            cv2.imshow('image', img)  # 사각형을 그린 이미지 표시
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        release_position = (x, y)  # 마우스를 뗀 좌표 저장
        rect_start = (ix, iy)
        rect_end = (x, y)  # 사각형의 시작과 끝 좌표 저장
        if mode:
            img = img_original.copy()  # 원본 이미지를 복사
            cv2.rectangle(img, rect_start, rect_end, (0, 0, 255), -1)  # 사각형 그리기
            cv2.imshow('image', img)  # 사각형을 그린 이미지 표시

In [30]:
# 트랙바 콜백 함수
def update_alpha(val):
    global alpha, rect_start, rect_end, img, img_original
    alpha = val / 10.0  # 트랙바 값을 0에서 1 사이의 값으로 변환
    img = img_original.copy()  # 원본 이미지를 복사
    if rect_start and rect_end:  # 사각형이 그려진 좌표가 있을 때만 투명도 조정
        overlay = img.copy()
        cv2.rectangle(overlay, rect_start, rect_end, (0, 0, 255), -1)  # 사각형 그리기
        # 알파 블렌딩을 통해 투명도를 적용
        cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0, img)
        cv2.imshow('image', img)

In [31]:
# 초기 설정
cv2.namedWindow('image')
cv2.setMouseCallback('image', draw_shape)

# 트랙바 생성 및 초기화
cv2.createTrackbar('value', 'image', 10, 10, update_alpha)  # 초기값 10 = 완전 불투명

while True:
    cv2.imshow('image', img)  # 이미지 표시

    # 텍스트 표시: 클릭한 좌표와 뗀 좌표가 정해졌다면
    if click_position and release_position:
        cv2.putText(img, f'Mouse Position ({click_position[0]}, {click_position[1]}) - ({release_position[0]}, {release_position[1]})', 
                    (10, 50), font, 0.7, (255, 255, 255), 1, cv2.LINE_AA)

    k = cv2.waitKey(1) & 0xFF
    if k == ord('m'):
        mode = not mode
    elif k == 27:  # Esc 키를 누르면 종료
        break

cv2.destroyAllWindows()